In [1]:
import os
import sys
import tensorflow                             as tf
import numpy                                  as np

import time
import shutil
import h5py
import seaborn                                as sns
import pandas                                 as pd
import matplotlib.pyplot                      as plt
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path

from tensorflow                           import keras

In [2]:
WORKSPACE_PATH    = os.getenv('WORKSPACE_PATH')  

SurQCTFldr        = WORKSPACE_PATH + '/SurQCT/surqct/'

TTran             = 10000.0  #[2500.0, 5000.0, 6000.0, 8000.0, 12000.0, 14000.0, 15000.0, 20000.0]

PathToLevelsFile  = [WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/O2_nd.csv']
KineticFldr       = WORKSPACE_PATH + '/Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_nondim/'

PathToDiatPot     = WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp'

In [3]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Input Module ...")

InputFile = SurQCTFldr + '/src/InputData/'
print("[SurQCT]:   Calling SurQCT with the PRESET Input File Located in " + InputFile )
sys.path.insert(0, InputFile)
from SurQCT_Input import inputdata

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading import read_levelsdata

#====================================================================================================================================


[SurQCT]: Loading Input Module ...
[SurQCT]:   Calling SurQCT with the PRESET Input File Located in /Users/sventuri/WORKSPACE//SurQCT/surqct//src/InputData/


In [4]:
#===================================================================================================================================
print("\n[SurQCT]: Initializing Input ...")
WORKSPACE_PATH             = os.getenv('WORKSPACE_PATH')  

InputData                  = inputdata(WORKSPACE_PATH, SurQCTFldr)

InputData.PathToLevelsFile = PathToLevelsFile
InputData.RatesType        = 'KExcit'
InputData.xVarsVec         = ['EVib','ERot','rMin','rMax','VMin','VMax','Tau','ri','ro'] #['EVib','ERot','ri','ro'] 
InputData.OtherVar         = '_Delta'
InputData.ApproxModel      = 'DotNet'

OtherVar                   = InputData.OtherVar
xVarsVec                   = InputData.xVarsVec

InputData.MultFact         = 1.e+9
MinValue                   = 1.e-16
#===================================================================================================================================


[SurQCT]: Initializing Input ...


In [5]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model

#===================================================================================================================================


[SurQCT]: Loading Final Modules ... 


In [6]:
#===================================================================================================================================
print('\n[SurQCT]: Initializing ML Model for KInel and Loading its Parameters ... ')

PathToRunFld              = SurQCTFldr + '/../' + InputData.RatesType + '/all_temperatures_nondim/KInel_Test' + str(InputData.NNRunIdx)
InputData.PathToDataFld   = PathToRunFld + '/Data/'                                                               
InputData.PathToParamsFld = PathToRunFld + '/Params/'                                                            

NN_KInel     = model(InputData, PathToRunFld, None, None)
NN_KInel.load_params(InputData.PathToParamsFld)


#    print('\n[SurQCT]: Initializing ML Model for KExch and Loading its Parameters ... ')
#
#    PathToRunFld              = SurQCTFldr + '/../' + InputData.RatesType + '/KExch_Test' + str(InputData.NNRunIdx)
#    InputData.PathToDataFld   = PathToRunFld + '/Data/'                                                               
#    InputData.PathToParamsFld = PathToRunFld + '/Params/'   
#
#    NN_KExch     = model(InputData, PathToRunFld, None, None)
#    NN_KExch.load_params(InputData.PathToParamsFld)

#===================================================================================================================================


[SurQCT]: Initializing ML Model for KInel and Loading its Parameters ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'rMin_i', 'rMax_i', 'VMin_i', 'VMax_i', 'Tau_i', 'ri_i', 'ro_i', 'TTran_i', 'EVib_Delta', 'ERot_Delta', 'rMin_Delta', 'rMax_Delta', 'VMin_Delta', 'VMax_Delta', 'Tau_Delta', 'ri_Delta', 'ro_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /Users/sventuri/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KInel_Test1/MyModel
[SurQCT]:   Loading ML Model Parameters from File:  /Users/sventuri/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KInel_Test1/Params//ModelCheckpoint/cp-4316.ckpt


In [7]:
Branch_KInel = keras.Model(NN_KInel.Model.inputs, NN_KInel.Model.layers[-4].output)
Branch_KInel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
tf.split (TFOpLambda)        [(None, 10), (None, 10)]  0         
_________________________________________________________________
normalization (Normalization (None, 10)                21        
_________________________________________________________________
Branch0_HL1 (Dense)          (None, 64)                704       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
Branch0_HL2 (Dense)          (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0     

In [8]:
Trunk_KInel = keras.Model(NN_KInel.Model.inputs, NN_KInel.Model.layers[-3].output)
Trunk_KInel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
tf.split (TFOpLambda)        [(None, 10), (None, 10)]  0         
_________________________________________________________________
normalization_1 (Normalizati (None, 10)                21        
_________________________________________________________________
Branch1_HL1 (Dense)          (None, 64)                704       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
Branch1_HL2 (Dense)          (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0   

In [9]:
#===================================================================================================================================
print('\n[SurQCT]: Generating Rate Matrix at Translational Temperature: T = ' + str(int(TTran)) + 'K')


#===================================================================================================================================
### Reading Levels Info of Initial and Final Molecules
Suffix       = ''

LevelsDataOrig     = pd.read_csv(InputData.PathToLevelsFile[0], header=0, skiprows=0)
vqn                = LevelsDataOrig['vqn'].to_numpy(int)
jqn                = LevelsDataOrig['jqn'].to_numpy(int)
LevelsData         = LevelsDataOrig[xVarsVec]
LevelsData         = LevelsData.astype('float64')
LevelsData.columns = [(VarName + Suffix) for VarName in LevelsData.columns]
NLevels            = LevelsData.shape[0]
LevelsData['EVib'] = np.log10(LevelsData['EVib'] + 1e-10)
LevelsData['Rot']  = np.log10(LevelsData['ERot']  + 1e-10)
LevelsData['Tau']  = np.log10(LevelsData['Tau'])
LevelsData['ro']   = np.log10(LevelsData['ro'])
#===================================================================================================================================


[SurQCT]: Generating Rate Matrix at Translational Temperature: T = 10000K


In [38]:
try:
    os.makedirs(KineticFldr)
except OSError as e:
    pass


PathToKineticFldr = KineticFldr + '/Test' + str(InputData.NNRunIdx) + '/T' + str(int(TTran)) + 'K'
try:
    os.makedirs(PathToKineticFldr)
except OSError as e:
    pass

#===================================================================================================================================
### Opening Files for Writing Rates
Branch_File = PathToKineticFldr + '/Branch.csv'
Trunk_File  = PathToKineticFldr + '/Trunk.csv'
Group_File  = PathToKineticFldr + '/Groups.csv'

In [11]:
ColumnNames = []
for i in range(64):
    ColumnNames.append('N'+str(i+1))
print(ColumnNames)

['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N20', 'N21', 'N22', 'N23', 'N24', 'N25', 'N26', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32', 'N33', 'N34', 'N35', 'N36', 'N37', 'N38', 'N39', 'N40', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N47', 'N48', 'N49', 'N50', 'N51', 'N52', 'N53', 'N54', 'N55', 'N56', 'N57', 'N58', 'N59', 'N60', 'N61', 'N62', 'N63', 'N64']


In [ ]:
# Getting Info on Diatomic Potential

In [12]:
LevelsDataNew         = pd.read_csv(PathToDiatPot, delim_whitespace=True, skiprows=15, header=None)
LevelsDataNew.columns = ['vqn','jqn','eint','egam','rmin','rmax','vmin','vmax',' tau','ri','ro']

LevelsDataNew         = LevelsDataNew.copy()
LevelsDataNew['eint'] = LevelsDataNew['eint'] * 27.211399
LevelsDataNew['vmin'] = LevelsDataNew['vmin'] * 27.211399
LevelsDataNew['vmax'] = LevelsDataNew['vmax'] * 27.211399

In [26]:
### Loop on Initial States
VirginFlg = True

#for iIdx in tqdm(range(1,NLevels+1), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
for iIdx in range(0, 1):    
    time.sleep(0.02)
    jIdxVec               = [iIdx]
    jNLevels              = len(jIdxVec)
    kIdxVec               = [iIdx]*jNLevels

    TTranVec              = np.ones((jNLevels))*TTran
    TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
    TTranDataTemp.index   = jIdxVec

    
    #########################################################################
    iLevelsDataTemp       = LevelsData.iloc[kIdxVec,:].copy()
    iLevelsDataTemp.index = jIdxVec

    jLevelsDataTemp       = LevelsData.iloc[jIdxVec,:].copy()        
    if (OtherVar == '_Delta'):
        jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
    else:
        jLevelsDataTemp   = jLevelsDataTemp
    jLevelsDataTemp.index = jIdxVec

    iLevelsData           = pd.concat([iLevelsDataTemp, TTranDataTemp], axis=1)
    iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    jLevelsData           = pd.concat([jLevelsDataTemp, TTranDataTemp], axis=1)
    jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    xTemp                 = pd.concat([iLevelsData, jLevelsData], axis=1)
    #########################################################################
    
    
    KBranch               = Branch_KInel.predict(xTemp[NN_KInel.xTrainingVar])
    TempData              = pd.DataFrame(LevelsDataNew.iloc[[iIdx]])
    DataTot               = pd.concat([TempData.reset_index(drop=True), pd.DataFrame(KBranch, columns=ColumnNames)], axis=1)
    
    if (VirginFlg):
        DataTot.to_csv(Branch_File, index=False)
        VirginFlg = False
    else:
        DataTot.to_csv(Branch_File, index=False, mode='a', header=False)

In [49]:
### Loop on Initial States
VirginFlg = True
OtherVar_ = 'j'

#for iIdx in tqdm(range(1000, 1001), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
for iIdx in tqdm([3440], desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
    time.sleep(0.02)
    jIdxVec               = np.arange(iIdx)
    jNLevels              = len(jIdxVec)
    kIdxVec               = [iIdx]*jNLevels

    TTranVec              = np.ones((jNLevels))*TTran
    TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
    TTranDataTemp.index   = jIdxVec

    
    #########################################################################
    iLevelsDataTemp       = LevelsData.iloc[kIdxVec,:].copy()
    iLevelsDataTemp.index = jIdxVec

    jLevelsDataTemp       = LevelsData.iloc[jIdxVec,:].copy()        
    if (OtherVar == '_Delta'):
        jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
    else:
        jLevelsDataTemp   = jLevelsDataTemp
    jLevelsDataTemp.index = jIdxVec

    iLevelsData           = pd.concat([iLevelsDataTemp, TTranDataTemp], axis=1)
    iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    jLevelsData           = pd.concat([jLevelsDataTemp, TTranDataTemp], axis=1)
    jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    xTemp                 = pd.concat([iLevelsData, jLevelsData], axis=1)
    #########################################################################
    
    
    
    #########################################################################
    iLevelsDataTemp       = LevelsDataOrig.iloc[kIdxVec,:].copy()
    iLevelsDataTemp.index = jIdxVec

    jLevelsDataTemp       = LevelsDataOrig.iloc[jIdxVec,:].copy()        
    if (OtherVar_ == '_Delta'):
        jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
    else:
        jLevelsDataTemp   = jLevelsDataTemp
    jLevelsDataTemp.index = jIdxVec

    iLevelsDataOrig           = pd.concat([iLevelsDataTemp, TTranDataTemp], axis=1)
    iLevelsDataOrig.columns   = [(VarName + '_i') for VarName in iLevelsDataOrig.columns]
    jLevelsDataOrig           = pd.concat([jLevelsDataTemp, TTranDataTemp], axis=1)
    jLevelsDataOrig.columns   = [(VarName + OtherVar_) for VarName in jLevelsDataOrig.columns]    
    #########################################################################

    
    
    KTrunk                = Trunk_KInel.predict(xTemp[NN_KInel.xTrainingVar])
    DataTot               = pd.concat([iLevelsDataOrig, jLevelsDataOrig, pd.DataFrame(KTrunk, columns=ColumnNames)], axis=1)
    
    if (VirginFlg):
        DataTot.to_csv(Trunk_File, index=False)
        VirginFlg = False
    else:
        DataTot.to_csv(Trunk_File, index=False, mode='a', header=False)

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 1/1 [00:00<00:00,  2.05it/s]


In [35]:
Data = pd.read_csv(Branch_File)[ColumnNames]
X    = Data.to_numpy()

In [45]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=150, random_state=0).fit(X)

LevelsDataNew['groups'] = kmeans.labels_
LevelsDataNew.to_csv(Group_File, index=False)